https://www.kaggle.com/sid321axn/amazon-alexa-reviews

In [1]:
import numpy as np
import pandas as pd
import nltk

In [2]:
import os
print(os.listdir("../alexa-ML"))

['.ipynb_checkpoints', 'amazon_alexa.tsv', 'Data Manipulation.ipynb', 'loan-prediction-analytics-vidhya-master', 'loan-train.csv', 'Sentiment Analysis.ipynb', 'smartphone-review.zip']


In [3]:
review = pd.read_csv('../alexa-ML/amazon_alexa.tsv', sep = '\t', encoding="utf8")

In [4]:
review.head(3)

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1


In [5]:
review.groupby('rating').describe()

feedback                                   
          count mean  std  min  25%  50%  75%  max
rating                                            
1         161.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
2          96.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
3         152.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
4         455.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0
5        2286.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0

In [6]:
review['verified_reviews'][2]

'Sometimes while playing a game, you can answer a question correctly but Alexa says you got it wrong and answers the same as you.  I like being able to turn lights on and off while away from home.'

In [7]:
import re
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

stop_words = stopwords.words("english")
lemma = WordNetLemmatizer()

corpus = []
for i in range(0, len(review)):
    rv = review['verified_reviews'][i]
    sent = sent_tokenize(rv)
    words = []
    for j in range(0, len(sent)):
        word = [w for w in sent if w.isalpha()]
        word = [w.lower() for w in word]
        word = [w for w in word if not w in stop_words]
        
#     sent = [word for word in sent if word.isalpha()]
#     sent = [w.lower() for w in sent]
#     sent = [w for w in sent if not w in stop_words]
#     sent = [lemma.lemmatize(w, 'v') for w in sent]
    corpus.append(sent)
        

In [10]:
corpus[2]

['Sometimes while playing a game, you can answer a question correctly but Alexa says you got it wrong and answers the same as you.',
 'I like being able to turn lights on and off while away from home.']

#### Bags of words

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features = 1500)
#X = cv.fit_transform(corpus).toarray()
X = CountVectorizer(tokenizer = lambda doc: doc, lowercase = False).fit_transform(corpus)
y = review.iloc[:, 4].values

In [52]:
review.groupby('feedback').describe()

rating                                             
           count      mean       std  min  25%  50%  75%  max
feedback                                                     
0          257.0  1.373541  0.484688  1.0  1.0  1.0  2.0  2.0
1         2893.0  4.737643  0.546544  3.0  5.0  5.0  5.0  5.0

#### Balance Labels

https://beckernick.github.io/oversampling-modeling/

#### SMOTE - Synthetic Minority Oversampling Technique
It creates synthetic observations of the minority class (bad feedbacks) by:
1. Finding the k-nearest-neighbors for minority class observations (finding similar observations)
2. Randomly choosing one of the k-nearest-neighbors and using it to create a similar, but randomly tweaked, new observation. 

#### Breakdown to Train & Test

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 25)

In [27]:
X_train.shape

(2520, 5392)

#### Random Forest

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [44]:
rf = RandomForestClassifier(n_estimators = 100, 
                            max_features = "auto",
                            random_state = 58)

In [45]:
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=58, verbose=0, warm_start=False)

In [47]:
y_pred = rf.predict(X_test)

In [48]:
accuracy_score(y_test, y_pred)

0.9428571428571428

In [49]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [50]:
print(cm)

[[ 13  36]
 [  0 581]]


#### Ada Boosting

In [51]:
from sklearn.ensemble import AdaBoostClassifier

#### Decision Tree

A decision tree is a tree where each node represents a feature (attribute), each link (branch) represents a decision (rule) and each lead represents an outcome (categorical or continuous value).

1. Easy to Understand - it mimics human level thinking 
2. Visually clear to see

Basic Algorithms to build a decision tree:
1. CART (Classification And Regression Trees) - uses Gini Index (classification) as metric.
2. IDS (Iterative Dichotomiser3) - uses Entropy function and Information Gain as metrics.

In [122]:
import re
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

stop_words = stopwords.words("english")
lemma = WordNetLemmatizer()

corpus = []
for i in range(0, len(review)):
    rv = review['verified_reviews'][i]
    sent = sent_tokenize(rv)
    words = []
    for j in range(0, len(sent)):
        token = word_tokenize(sent[j])
        word = [word for word in token if word.isalpha()]
        word = [w.lower() for w in word]
        word = [w for w in word if not w in stop_words]
        word = [lemma.lemmatize(w, 'v') for w in word]
        words.append(word)
        
    corpus.append(words)

In [123]:
corpus[2]

[['sometimes',
  'play',
  'game',
  'answer',
  'question',
  'correctly',
  'alexa',
  'say',
  'get',
  'wrong',
  'answer'],
 ['like', 'able', 'turn', 'light', 'away', 'home']]

Stemming & Lemmazation

Compared with stemming, lemmazatizer, a tool from Natural Language Processing which does full morphological analysis to accurately identify the lemma for each word. Doing full morphological analysis produces at most very modest benefits fro retrieval. 